In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
from datetime import datetime


In [11]:
# ===================== CẤU HÌNH =====================
CATEGORIES = {
    "doanh-nghiep": "https://cafef.vn/doanh-nghiep.chn",
    "thi-truong": "https://cafef.vn/thi-truong.chn",
    "chung-khoan": "https://cafef.vn/chung-khoan.chn",
    "bat-dong-san": "https://cafef.vn/bat-dong-san.chn",
}
MAX_LOAD_MORE = 10   # số lần click "Xem thêm"
CRAWL_DATE = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
# ===================================================

In [12]:
# --- Setup Chrome ---
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [13]:
all_data = []

# --- Duyệt từng chuyên mục ---
for cat_name, url in CATEGORIES.items():
    print(f"\n📰 Đang crawl chuyên mục: {cat_name}")
    driver.get(url)
    wait = WebDriverWait(driver, 10)

    # Click nút "Xem thêm" để load thêm bài
    for i in range(MAX_LOAD_MORE):
        try:
            # btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.view-more-news")))
            # driver.execute_script("arguments[0].click();", btn)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
        except Exception:
            break

    time.sleep(1)
    articles = driver.find_elements(By.CSS_SELECTOR, "div.tlitem")

    for a in articles:
        try:
            title_elem = a.find_element(By.CSS_SELECTOR, "h3 a")
            title = title_elem.text.strip()
            link = title_elem.get_attribute("href")

            desc = ""
            try:
                desc_elem = a.find_element(By.CSS_SELECTOR, "h3 + div")
                desc = desc_elem.text.strip()
            except:
                pass

            time_str = ""
            try:
                time_elem = a.find_element(By.CSS_SELECTOR, ".pdate")
                time_str = time_elem.text.strip()
            except:
                pass

            all_data.append({
                "category": cat_name,
                "title": title,
                "link": link,
                "description": desc,
                "time_posted": time_str,
                "crawl_date": CRAWL_DATE
            })
        except Exception as e:
            print("Lỗi parse bài:", e)

print(f"\n✅ Tổng số bài thu được: {len(all_data)}")

# --- Đóng trình duyệt ---
driver.quit()

# --- Lấy nội dung chi tiết từng bài ---
print("\n🧠 Đang tải nội dung chi tiết từng bài...")
for item in all_data:
    try:
        html = requests.get(item["link"], timeout=10).text
        soup = BeautifulSoup(html, "html.parser")
        content = " ".join([p.text.strip() for p in soup.select("div.detail-content p")])
        item["content"] = content
        time.sleep(0.5)
    except Exception as e:
        item["content"] = ""
        print(f"Lỗi tải nội dung: {e}")

# --- Xuất ra CSV ---
df = pd.DataFrame(all_data)
df.to_csv("cafef_full_data_2.csv", index=False, encoding="utf-8-sig")

print("\n🎯 Crawl hoàn tất!")
print(f"📁 File lưu tại: cafef_full_data.csv")



📰 Đang crawl chuyên mục: doanh-nghiep

📰 Đang crawl chuyên mục: thi-truong

📰 Đang crawl chuyên mục: chung-khoan

📰 Đang crawl chuyên mục: bat-dong-san

✅ Tổng số bài thu được: 192

🧠 Đang tải nội dung chi tiết từng bài...

🎯 Crawl hoàn tất!
📁 File lưu tại: cafef_full_data.csv
